## 1 Water pump prediction

### 1.1 Library insertion

In [35]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import (
    AdaBoostClassifier,
    BaggingClassifier,
    RandomForestClassifier,
)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier

### 1.2 Data insertion

In [39]:
training_data_value = pd.read_csv("Water-Pump-Prediction/Data/Training_set_values.csv")
training_data_label = pd.read_csv("Water-Pump-Prediction/Data/Training_set_labels.csv")
testing_data = pd.read_csv("Water-Pump-Prediction/Data/Test_set_values.csv")

# merge data
Merged_training_data = pd.merge(training_data_value,training_data_label, on="id")
All_data = pd.concat([testing_data, Merged_training_data])
All_data.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
0,50785,0.0,4/2/2013,Dmdd,1996,DMDD,35.290799,-4.059696,Dinamu Secondary School,0,...,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,other,other,NaN
1,51630,0.0,4/2/2013,Government Of Tanzania,1569,DWE,36.656709,-3.309214,Kimnyak,0,...,soft,good,insufficient,insufficient,spring,spring,groundwater,communal standpipe,communal standpipe,NaN
2,17168,0.0,1/2/2013,NaN,1567,NaN,34.767863,-5.004344,Puma Secondary,0,...,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,other,other,NaN
3,45559,0.0,22/1/2013,Finn Water,267,FINN WATER,38.058046,-9.418672,Kwa Mzee Pange,0,...,soft,good,dry,dry,shallow well,shallow well,groundwater,other,other,NaN
4,49871,500.0,27/3/2013,Bruder,1260,BRUDER,35.006123,-10.950412,Kwa Mzee Turuka,0,...,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,NaN


### 1.X Support function

In [43]:
# support function for numerical data, for analysis whole dataframe
def numerical_analysis(dataFrame):
  
  # Only select numerical columns
  numerical = dataFrame.select_dtypes(include = ['int', 'Int64', 'float']).columns.tolist()
  
  #data type
  data_types = pd.DataFrame(dataFrame[numerical].dtypes,columns=['Data Type'])

  # number of data count
  count_val = pd.DataFrame(columns=['Count'])

  for row in list(dataFrame[numerical].columns.values):
      count_val.loc[row] = [dataFrame[numerical][row].count()]
    
  # missing data
  missing_data = pd.DataFrame(dataFrame[numerical].isnull().sum(),columns=['Missing Values'])

  # minimum value
  maximum_val = pd.DataFrame(columns=['Maximum'])

  for row in list(dataFrame[numerical].columns.values):
      maximum_val.loc[row] = [dataFrame[numerical][row].max()]
    
  # maximum value
  minimum_val = pd.DataFrame(columns=['Minimum'])

  for row in list(dataFrame[numerical].columns.values):
      minimum_val.loc[row] = [dataFrame[numerical][row].min()]
    
  # mean value
  mean_val= pd.DataFrame(columns=['Mean'])

  for row in list(dataFrame[numerical].columns.values):
      mean_val.loc[row] = [dataFrame[numerical][row].mean()]

  # mode
  mode_val = pd.DataFrame(columns=['Mode'])

  for row in list(dataFrame[numerical].columns.values): 
    mode_val.loc[row] = [dataFrame[numerical][row].mode()[0]]
    mode_0 = mode_val.loc[row]
  
  # mode frequency
  count_f_list = []

  for row in numerical:
    mode_0 = dataFrame[row].mode().iat[0]
    count_f = dataFrame[row].value_counts()[mode_0]
    count_f_list.append(count_f)

  count_f_list = np.array(count_f_list)
    
  # First quartile
  IqR_1 = pd.DataFrame(columns=['1st Quartile'])

  for row in list(dataFrame[numerical].columns.values):
      IqR_1.loc[row] = [dataFrame[numerical][row].quantile(0.25)]
    
  # Median
  median = pd.DataFrame(columns=['Median'])

  for row in list(dataFrame[numerical].columns.values):
    median.loc[row] = [dataFrame[numerical][row].quantile(0.5)]
    
  # Third quartile
  IqR_3 = pd.DataFrame(columns=['3rd Quartile'])

  for row in list(dataFrame[numerical].columns.values):
    IqR_3.loc[row] = [dataFrame[numerical][row].quantile(0.75)]

  # Unique values
  unique_val = pd.DataFrame(columns=['Unique Values'])

  for row in list(dataFrame[numerical].columns.values):
    unique_val.loc[row] = [dataFrame[numerical][row].nunique()]
  
    
  # join columns 
  result = data_types.join(count_val).join(missing_data).join(unique_val).join(minimum_val).join(maximum_val).join(mean_val).join(IqR_1).join(median).join(IqR_3).join(mode_val)
  
  # percentage missing
  result['Missing in %'] = (result['Missing Values'] / len(dataFrame[numerical]) *100)
  
  # alternative mode calculation
  result['Mode freq'] = count_f_list 
  result['Mode in %'] = (result['Mode freq'] / result['Count']*100).astype('float')


  # change order of columns
  result = result[['Data Type', 'Count', 'Missing Values', 'Missing in %','Unique Values', 'Mean', 'Minimum', '1st Quartile', 'Median', '3rd Quartile', 'Maximum', 'Mode', 'Mode freq','Mode in %']]

  #round
  result[['Missing in %', 'Mean', 'Minimum', '1st Quartile', 'Median', '3rd Quartile', 'Maximum','Mode freq','Mode in %','Mode']] =  result[['Missing in %', 'Mean', 'Minimum', '1st Quartile', 'Median', '3rd Quartile', 'Maximum', 'Mode freq','Mode in %','Mode']].round(1)
  
   
  #rreturn report
  return(result)

In [58]:
num_analysis = numerical_analysis(All_data)
num_analysis

,Data Type,Count,Missing Values,Missing in %,Unique Values,Mean,Minimum,1st Quartile,Median,3rd Quartile,Maximum,Mode,Mode freq,Mode in %
id,int64,74250,0,0.0,74250,37124.5,0.0,18562.2,37124.5,55686.8,74249.0,0.0,1,0.0
amount_tsh,float64,74250,0,0.0,102,318.7,0.0,0.0,0.0,20.0,350000.0,0.0,52049,70.1
gps_height,int64,74250,0,0.0,2456,665.7,-90.0,0.0,364.0,1317.0,2777.0,0.0,25649,34.5
longitude,float64,74250,0,0.0,71870,34.1,0.0,33.1,34.9,37.2,40.3,0.0,2269,3.1
latitude,float64,74250,0,0.0,71869,-5.7,-11.6,-8.5,-5.0,-3.3,-0.0,-0.0,2269,3.1
num_private,int64,74250,0,0.0,68,0.5,0.0,0.0,0.0,0.0,1776.0,0.0,73299,98.7
region_code,int64,74250,0,0.0,27,15.3,1.0,5.0,12.0,17.0,99.0,11.0,6608,8.9
district_code,int64,74250,0,0.0,20,5.6,0.0,2.0,3.0,5.0,80.0,1.0,15299,20.6
population,int64,74250,0,0.0,1128,180.8,0.0,0.0,25.0,215.0,30500.0,0.0,26834,36.1
construction_year,int64,74250,0,0.0,55,1298.5,0.0,0.0,1986.0,2004.0,2013.0,0.0,25969,35.0


In [51]:
def categorical_analysis(dataFrame):

    # Select categorical columns
    cat_columns = dataFrame.select_dtypes(include='object').columns.tolist()

    # Data types
    data_types = pd.DataFrame(dataFrame[cat_columns].dtypes,columns=['Data Type'])


    # Missing data
    missing_df = pd.DataFrame(dataFrame[cat_columns].isnull().sum(),columns=['Missing Values'])

    # Record counts
    records_df = pd.DataFrame(columns=['Records'])

    for col in list(dataFrame[cat_columns].columns.values):
        records_df.loc[col] = [dataFrame[cat_columns][col].count()]

    # Unique values
    unique_val = pd.DataFrame(columns=['Unique Values'])
    for col in list(dataFrame[cat_columns].columns.values):
        unique_val.loc[col] = [dataFrame[cat_columns][col].nunique()]

    # Mode
    mode_df = pd.DataFrame(columns=['Mode'])

    for col in list(dataFrame[cat_columns].columns.values):
        mode_df.loc[col] = [dataFrame[cat_columns][col].mode()[0]]

    # Mode frequency
    mode_freq_list = []
    for col in cat_columns:
        mode_val = dataFrame[col].mode().iat[0]
        freq = dataFrame[col].value_counts()[mode_val]
        mode_freq_list.append(freq)
    mode_freq_array = np.array(mode_freq_list)

    # Create data quality report
    result = data_types.join(missing_df).join(records_df).join(unique_val).join(mode_df)
    result['Mode freq'] = mode_freq_array
    result['Mode in %'] = (result['Mode freq'] / result['Records'] * 100).astype('float')
    result['Missing in %'] = (result['Missing Values'] / len(dataFrame[cat_columns]) * 100)

    # Change order of columns
    result = result[['Data Type', 'Records', 'Unique Values', 'Missing Values', 'Missing in %', 'Mode', 'Mode freq', 'Mode in %']]
    result[['Missing in %', 'Mode in %']] = result[['Missing in %', 'Mode in %']].round(1)

    # Return report
    return result


In [59]:
cat_analysis = categorical_analysis(All_data)
cat_analysis

,Data Type,Records,Unique Values,Missing Values,Missing in %,Mode,Mode freq,Mode in %
date_recorded,object,74250,687,0,0.0,2011-03-15,572,0.8
funder,object,69743,2139,4507,6.1,Government Of Tanzania,11299,16.2
installer,object,69718,2410,4532,6.1,DWE,21751,31.2
wpt_name,object,74248,45683,2,0.0,none,4440,6.0
basin,object,74250,9,0,0.0,Lake Victoria,12871,17.3
subvillage,object,73780,21425,470,0.6,Shuleni,646,0.9
region,object,74250,21,0,0.0,Iringa,6599,8.9
lga,object,74250,125,0,0.0,Njombe,3128,4.2
ward,object,74250,2098,0,0.0,Igosi,386,0.5
public_meeting,object,70095,2,4155,5.6,True,63749,90.9


In [53]:
# check duplicate 
def check_duplicate(dataFrame):
  duplicates = dataFrame.duplicated(keep = 'first').sum()
  print("No. duplicates: {}".format(duplicates))
  print("Total counts: {}".format(len(dataFrame)))
  print("Duplicated in %: {}".format((duplicates/len(dataFrame))*100))

In [56]:
check_duplicate(All_data)

No. duplicates: 0
Total counts: 74250
Duplicated in %: 0.0


## 2 EDA - Preview

### 2.1 Duplication

In [57]:
check_duplicate(All_data)

No. duplicates: 0
Total counts: 74250
Duplicated in %: 0.0


### 2.X 

## 3 EDA - Data Features

### 3.1 Amount_tsh

In [60]:
num_analysis.loc[['amount_tsh']]

,Data Type,Count,Missing Values,Missing in %,Unique Values,Mean,Minimum,1st Quartile,Median,3rd Quartile,Maximum,Mode,Mode freq,Mode in %
amount_tsh,float64,74250,0,0.0,102,318.7,0.0,0.0,0.0,20.0,350000.0,0.0,52049,70.1


In [64]:
All_data['amount_tsh'].isna().sum()

0

In [65]:
All_data['amount_tsh'] = All_data['amount_tsh'].replace(0,All_data['amount_tsh'].mean())

In [68]:
All_data['amount_tsh'].mode()

0    318.685704
Name: amount_tsh, dtype: float64

The most occured value is 0 and it was over 70% in whole data. We can assume that it contains some missing value.
It could be replaced by mean values or drop this feature after experiment.

### 3.2 date_recorded